# Data Preprocessing notebook
* Data Processing
    * define function to load raw json format by partitioning to prevent out of memory error 
    * clean dataset (special characters and html content) and drop redundant columns (except dropping NA as some columns could be useful in other tasks)
    * deduplication
    * add year column
    * output to csv (`product.csv` and `review.csv`)

* Data Manipulation
    * load `product.csv` and `review.csv` again
    * join two columns from the product dataset to review the dataset to add product main category (`main_cat`) and price (`price`) info to review the dataset for preliminary analysis
    * output csv `product.csv`

Output of this notebook: `product.csv` and `review.csv`

## Data Processing

In [1]:
import json
from tqdm import tqdm
import pandas as pd
import os

import html
import re 
import numpy as np

C:\Users\rzp98\AppData\Local\Temp\ipykernel_5708\3259503239.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


data source https://nijianmo.github.io/amazon/index.html

In [ ]:
os.path.getsize("meta_Electronics.json") 

In [ ]:
os.path.getsize("Electronics.json")

In [ ]:
# define a custom function which support loading partition to prevent out of memory error 

def get_row_counts(file_path):
    with open(file_path, "r") as file:
        row_counts = 0
        for rows in file:
            row_counts += 1
    return row_counts

def load_json_to_df(file_path, partition = 1):
    row_counts = get_row_counts(file_path)
    with open(file_path, "r") as file:
        # start from first partition
        rows_rolling = 0
        json_object = []
        for rows in tqdm(file):
            if rows_rolling > row_counts / partition:
                print(f"Total rows processed {rows_rolling}/{row_counts}")
                break
            else:
                json_obj = json.loads(rows.strip())
                json_object.append(json_obj)
                rows_rolling += 1
                if rows_rolling % 100000 == 0:
                    print(f"Processing row: {rows_rolling}/{row_counts}")
                if rows_rolling == row_counts:
                    print(f"All rows loaded to json objects: {rows_rolling}/{row_counts}")
                    
    print("pushing json objects to pd.df")
    return pd.DataFrame(json_object)

### Product data (meta_Electronics dataset)

In [ ]:
df_product = load_json_to_df(file_path="meta_Electronics.json", partition = 1)

In [ ]:
df_product.info()

#### df_product: data definition and keep/drop rule
* category: product category and sub-catecory information - `*DROP*`
* tech1: primary product specification informtion in HTML format - `*DROP*`
* description: product description - `*KEEP*`
* fit: this column contains wearable electronic product information in HTML format - `*DROP*`
* title: product title - `*KEEP*`
* also_buy: list of asin of product which customer who bought this also bought - `*DROP*`
* tech2: secondary product specification informtion in HTML format -`*DROP*`
* brand: brand of product - `*KEEP*`
* feature: product features - `*DROP*`
* rank: product rank, need to be clearned as formating is not unified - `*KEEP*`
* also_view: list of asin of product which customer who viewed this also viewed - `*DROP*`
* main_cat: main product category - `*KEEP*`
* similar_item: similar item, html content - `*DROP*`
* date: product date - `*KEEP*`
* price: price information - `*KEEP*`
* asin: product id - `*KEEP*`
* imageURL: URL of product image - `*DROP*`
* imageURLHighRes: URL of product image - `*DROP*`
* details: - `*DROP*`


In [ ]:
columns_to_drop_price = [
    'category', 'tech1', 'fit', 'also_buy', 'tech2', 
    'feature', 'also_view', 'similar_item', 'imageURL', 
    'imageURLHighRes', 'details'
    ]
# Dropping the redundant columns
df_product = df_product.drop(columns=columns_to_drop_price)

In [ ]:
df_product.info()

#### Data Clearning (special characters and html content)

In [ ]:
## Define function for description and value 

def clean_description(description):
    # Check if the input is a list, and convert it to a string if so
    if isinstance(description, list):
        # Join the list elements into a single string separated by spaces
        description = ' '.join(description)
    
    description = html.unescape(description)
    # Remove HTML tags
    description = re.sub('<.*?>', ' ', description)
    # Replace non-alphanumeric characters with spaces
    description = re.sub('[^0-9a-zA-Z]+', ' ', description)
    # Normalise whitespace to single space
    description = re.sub(r'\s+', ' ', description)
    # Trim leading and trailing spaces
    description = description.strip()
    # Convert to lowercase
    description = description.lower()
    
    return description


def clean_main_cat(main_cat):
    # Convert any HTML entities to their corresponding characters
    main_cat = html.unescape(main_cat)
    # amazon fashion category is compromised contain url so normalise it to just amazon fashion
    main_cat = main_cat.replace('AMAZON FASHION', 'Amazon Fashion')
    
    return main_cat

In [ ]:
## Start cleaning

# Removing all $ signs from the 'price' column
df_product['price'] = df_product['price'].str.replace('$', '', regex=False)

# Convert price column to correct dtype to save memory
df_product['price'] = pd.to_numeric(df_product['price'], errors='coerce')

# Clean description and value
df_product['description'] = df_product['description'].apply(clean_description)
df_product['main_cat'] = df_product['main_cat'].apply(clean_main_cat)



In [ ]:
df_product.info()

In [ ]:
# drop duplicated asin
df_product = df_product.drop_duplicates(subset=['asin'])

In [ ]:
df_product.info()


Some columns were delibertly left out to be untouched to maintain the integrity of column asin
* description - cleaned to just alphanumeric text
* title - cleaned
* brand - cleaned
* rank - untouched, contain multiple category rankings
* main_cat - cleaned
* date - untouched, some dates are still compromise
* price - cleaned
* asin - cleaned

In [ ]:
# output to CSV
df_product.to_csv('product.csv', index=False)
del df_product

### Review data (Electronics dataset)

In [ ]:
df_review = load_json_to_df(file_path="Electronics.json", partition = 1)

In [ ]:
df_review.info()

In [ ]:
df_review.head()

### df_review: data definition and keep/drop rule
* overall: rating of the product                                               - `*KEEP*`
* verified: boolean                                                            - `*KEEP*`
* reviewTime: time of the review (raw)                                         - `*KEEP*`
* reviewerID: ID of the reviewer, e.g. A2SUAM1J3GNN3B                          - `*DROP*`
* asin: ID of the product, e.g. 0000013714                                     - `*KEEP*`
* style: a dictionary of the product metadata, e.g., "Format" is "Hardcover"   - `*DROP*`
* reviewerName: name of the reviewer                                           - `*DROP*`
* reviewText: text of the review                                               - `*KEEP*`
* summary: summary of the review                                               - `*DROP*`
* unixReviewTime: time of the review (Unix time format), e.g. 1026864000	   - `*DROP*`
* vote: helpful votes of the review, e.g. NaN or int                           - `*KEEP*`
* image: images that users post after they have received the product           - `*KEEP*`

In [ ]:
columns_to_drop_review = [
    'reviewerID', 'style', 'reviewerName', 'summary', 'unixReviewTime'
    ]
# Dropping the redundant columns
df_review = df_review.drop(columns=columns_to_drop_review)
df_review.info()

In [ ]:
# convert image url to boolean
df_review['image'] = df_review['image'].notna() & (df_review['image'] != '')
df_review['image'].value_counts()

In [ ]:
# convert data type
df_review['reviewTime'] = pd.to_datetime(df_review['reviewTime'], format='%m %d, %Y')

In [ ]:
# add year column
df_review['Year'] = df_review['reviewTime'].dt.year

In [ ]:
# drop duplicated rows
df_review = df_review.drop_duplicates()

In [ ]:
# drop NA in important columns
df_review.dropna(subset=['overall'], inplace=True)
df_review.dropna(subset=['reviewText'], inplace=True)

In [ ]:
df_review['reviewText'].isna().sum()

In [ ]:
df_review.info()

In [ ]:
df_review.head()

In [ ]:
# output to CSV
df_review.to_csv('review.csv', index=False)
del df_review

# Data Manipulation

Join two columns from the product dataset to review the dataset to add product main category (`main_cat`) and price (`price`) info to review the dataset for preliminary analysis

In [2]:
# load the cleaned data
df_review = pd.read_csv('review.csv')
df_product = pd.read_csv('product.csv')
# the warning was caused by uncleaned and mixed datatype in vote column, we will deal with this when we need vote data

C:\Users\rzp98\AppData\Local\Temp\ipykernel_5708\1604761310.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_review = pd.read_csv('review.csv')


In [3]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20567031 entries, 0 to 20567030
Data columns (total 8 columns):
 #   Column      Dtype  
---  ------      -----  
 0   overall     float64
 1   verified    bool   
 2   reviewTime  object 
 3   asin        object 
 4   reviewText  object 
 5   vote        object 
 6   image       bool   
 7   Year        int64  
dtypes: bool(2), float64(1), int64(1), object(4)
memory usage: 980.7+ MB


In [4]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 756077 entries, 0 to 756076
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   description  643149 non-null  object 
 1   title        756060 non-null  object 
 2   brand        750665 non-null  object 
 3   rank         756077 non-null  object 
 4   main_cat     751940 non-null  object 
 5   date         643586 non-null  object 
 6   price        293928 non-null  float64
 7   asin         756077 non-null  object 
dtypes: float64(1), object(7)
memory usage: 46.1+ MB


In [5]:
# the previous dropna might not detect all white spaces and some other special characters
# so we load the data again from csv and drop the NA again
df_review['reviewText'].isna().sum()

667

In [6]:
df_product['asin'].isna().sum()

0

In [7]:
df_review.dropna(subset=['reviewText'], inplace=True)
df_review['reviewText'].isna().sum()

0

In [8]:
df_review = df_review.merge(df_product[['asin', 'price']], on='asin', how='left')
df_review = df_review.merge(df_product[['asin', 'main_cat']], on='asin', how='left')

In [9]:
df_review['reviewText'].isna().sum()

0

In [10]:
df_review.head()

,overall,verified,reviewTime,asin,reviewText,vote,image,Year,price,main_cat
0,5.0,True,2002-07-17,0060009810,This was the first time I read Garcia-Aguilera...,NaN,False,2002,11.49,Books
1,5.0,False,2002-07-06,0060009810,"As with all of Ms. Garcia-Aguilera's books, I ...",NaN,False,2002,11.49,Books
2,5.0,False,2002-07-03,0060009810,I've not read any of Ms Aguilera's works befor...,2,False,2002,11.49,Books
3,4.0,False,2002-06-30,0060009810,This romance novel is right up there with the ...,3,False,2002,11.49,Books
4,5.0,False,2002-06-28,0060009810,Carolina Garcia Aguilera has done it again. S...,NaN,False,2002,11.49,Books


In [11]:
df_review.to_csv('review.csv', index=False)